In [1]:
from tensorflow import keras
import pydicom
import numpy as np
import tensorflow as tf
import os
from functools import partial, update_wrapper

In [2]:
#these are GPU options, that were required for my computing device
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [3]:
from keras.models import Model, load_model
from keras.layers import Input, Flatten, Dense, concatenate, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, Conv2DTranspose, Conv3DTranspose
from keras.layers import Activation, add, multiply, Lambda
from keras.layers import AveragePooling2D, AveragePooling3D, average, UpSampling2D, UpSampling3D, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras.initializers import glorot_normal, random_normal, random_uniform
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.losses import binary_crossentropy

Using TensorFlow backend.


In [4]:
image_path=''

In [5]:
def wrapped_partial(func, *args, **kwargs):
    partial_func = partial(func, *args, **kwargs)
    update_wrapper(partial_func, func)
    return partial_func


In [6]:
def dsc(y_true, y_pred, args):
    smooth = args.smooth
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred, args):
    loss = 1 - dsc(y_true, y_pred, args)
    return loss


def tp(y_true, y_pred, args):
    smooth = args.smooth
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pos = K.round(K.clip(y_true, 0, 1))
    tp = (K.sum(y_pos * y_pred_pos) + smooth) / (K.sum(y_pos) + smooth)
    return tp


def tn(y_true, y_pred, args):
    smooth = args.smooth
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    tn = (K.sum(y_neg * y_pred_neg) + smooth) / (K.sum(y_neg) + smooth)
    return tn


def fp(y_true, y_pred, args):
    smooth = args.smooth
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    fp = (K.sum(y_neg * y_pred_pos) + smooth) / (K.sum(y_pred) + smooth)
    return fp


def fn(y_true, y_pred, args):
    smooth = args.smooth
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    fn = (K.sum(y_pos * y_pred_neg) + smooth) / (K.sum(y_pred_neg) + smooth)
    return fn


def multiloss(y_true, y_pred, args):
    if args.loss_type == 1:
        return focal_tversky(y_true, y_pred, args)
    else:
        return dice_loss(y_true, y_pred, args)


In [7]:
def expand_as_3d(tensor, rep, name):
    my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=4), arguments={'repnum': rep},
                       name='psi_up' + name)(tensor)
    return my_repeat


def AttnGatingBlock3D(x, g, inter_shape, name):
    '''
    Analogous implementation of the 3D attention gate used in the Attention U-Net 3D.
    '''
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv3D(inter_shape, (2, 2, 2), strides=(2, 2, 2), padding='same', name='xl' + name)(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv3D(inter_shape, (1, 1, 1), padding='same')(g)
    upsample_g = Conv3DTranspose(inter_shape, (3, 3, 3), strides=(
    shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2], shape_theta_x[3] // shape_g[3]), padding='same',
                                 name='g_up' + name)(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv3D(1, (1, 1, 1), padding='same', name='psi' + name)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling3D(
        size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2], shape_x[3] // shape_sigmoid[3]))(
        sigmoid_xg)  # 32

    upsample_psi = expand_as_3d(upsample_psi, shape_x[4], name)
    y = multiply([upsample_psi, x], name='q_attn' + name)

    result = Conv3D(shape_x[4], (1, 1, 1), padding='same', name='q_attn_conv' + name)(y)
    result_bn = BatchNormalization(name='q_attn_bn' + name)(result)
    return result_bn


def UnetConv3D(input, outdim, is_batchnorm, name):
    '''
    Analogous implementation of the pair of convolutional layers used by the U-Net 3D.
    '''
    x = Conv3D(outdim, (3, 3, 3), strides=(1, 1, 1), kernel_initializer="glorot_normal", padding="same",
               name=name + '_1')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_1_bn')(x)
    x = Activation('relu', name=name + '_1_act')(x)

    x = Conv3D(outdim, (3, 3, 3), strides=(1, 1, 1), kernel_initializer="glorot_normal", padding="same",
               name=name + '_2')(x)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_2_bn')(x)
    x = Activation('relu', name=name + '_2_act')(x)
    return x


def UnetGatingSignal3D(input, is_batchnorm, name):
    '''
    Implementation of the gating signal appearing in the upsampling branch of the Attention U-Net 3D:
    simply a 1x1 convolution followed by batch normalization and ReLU.
    '''
    shape = K.int_shape(input)
    x = Conv3D(shape[4] * 1, (1, 1, 1), strides=(1, 1, 1), padding="same", kernel_initializer="glorot_normal",
               name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name=name + '_act')(x)
    return x


def tiny_attn_unet3D(opt, input_size, args):
    '''
    Analogue of the above defined attn_unet3D with less layers, resulting in a smaller U shape.
    '''
    inputs = Input(shape=input_size)
    conv1 = UnetConv3D(inputs, 8*args.kernel_power, is_batchnorm=True, name='conv1')
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    pool1 = Dropout(args.dropout)(pool1)

    conv2 = UnetConv3D(pool1, 8*args.kernel_power, is_batchnorm=True, name='conv2')
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    pool2 = Dropout(args.dropout)(pool2)

    center = UnetConv3D(pool2, 16*args.kernel_power, is_batchnorm=True, name='center')

    g3 = UnetGatingSignal3D(center, is_batchnorm=True, name='g3')
    attn3 = AttnGatingBlock3D(conv2, g3, 8*args.kernel_power, '_3')
    up3 = concatenate([Conv3DTranspose(8*args.kernel_power, (3, 3, 3), strides=(2, 2, 2), padding='same', activation='relu',
                                       kernel_initializer="glorot_normal")(center), attn3], name='up3')

    up4 = concatenate([Conv3DTranspose(8*args.kernel_power, (3, 3, 3), strides=(2, 2, 2), padding='same', activation='relu',
                                       kernel_initializer="glorot_normal")(up3), conv1], name='up4')

    mask_1 = Conv3D(1, (1, 1, 1), activation='sigmoid', name='mask_1')(up4)
    mask_2 = Conv3D(1, (1, 1, 1), activation='sigmoid', name='mask_2')(up4)
    dif = Conv3D(1, (1, 1, 1), activation='sigmoid', name='dif')(up4)

    model = Model(inputs=[inputs], outputs=[mask_1, mask_2, dif])
    model.compile(optimizer=opt,
                  loss=[wrapped_partial(dice_loss, args=args), wrapped_partial(dice_loss, args=args),
                        wrapped_partial(multiloss, args=args)],
                  loss_weights=[0.1, 0.1, 0.8],
                  metrics=[[wrapped_partial(dsc, args=args)], [wrapped_partial(dsc, args=args)],
                           [wrapped_partial(dsc, args=args), wrapped_partial(tp, args=args),
                            wrapped_partial(tn, args=args)]])
    return model

In [8]:
arguments = {'alpha': 0.6635482209772863, 'dropout': 0.0022118370088630712, 'gamma': 0.7940638705632237, 'kernel_power': 8, 'learning_rate': 0.00015099469943923109, 'loss_type': 0, 'smooth': 1, 'unet_type': 2}

In [9]:
class arg():
    def __init__(self):
        self.alpha=0.7
        self.gamma=0.75
        self.loss_type=1 #1 for focal tversky
        self.unet_type=0 #1 for attn unet
        self.smooth=1
        self.learning_rate=0.0001

In [10]:
args=arg()
args.loss_type=arguments['loss_type']
args.unet_type=arguments['unet_type']
args.smooth=arguments['smooth']
args.alpha=arguments['alpha']
args.gamma=arguments['gamma']
args.learning_rate=arguments['learning_rate']
args.dropout=arguments['dropout']
args.kernel_power=arguments['kernel_power']


In [11]:
model = tiny_attn_unet3D(Adam(learning_rate=args.learning_rate), (24,32,32,1),args)
model.load_weights('model_internal_dataset.h5')

In [12]:
def full_cta_model_evaluation(image, model, z_stride, which_prediction):
    '''
    Helper function, which given the original ct image of a plaque will give back the masks for the DL segmentation.
    '''
    # Shape of original images
    size_X = image.shape[2]
    size_Y = image.shape[1]
    size_Z = image.shape[0]

    image_paded = np.zeros((size_Z + 24,
                            size_Y,
                            size_X))

    image_paded[:size_Z, :size_Y, :size_X] = image / 512

    prediction_array = np.zeros((size_Z + 24,
                                 size_Y,
                                 size_X))

    coverage_array = np.zeros((size_Z + 24,
                               size_Y,
                               size_X))

    # Containers for batch predictions
    patch_boundaries_list = []
    counter = 0

    # Stride along Z axis:  
    for z_start in range(0, prediction_array.shape[2], z_stride):
        z_end = z_start + 24
        if (np.count_nonzero(image[z_start:z_end, :, :]) > 1):
            patch_boundaries_list.append([z_start, z_end])
    for patch_index in range(0, len(patch_boundaries_list)):
        # patch_boundaries in current batch
        temporal_boundaries = patch_boundaries_list[patch_index]
        temp_patches = []
        # Extracting patches for prediction
        current_patch = image_paded[temporal_boundaries[0]:temporal_boundaries[1],
                        16:48,
                        16:48]
        current_patch = np.expand_dims(current_patch, axis=0)
        # Updating prediction_array and coverage_array
        predicted_patch = model.predict(np.expand_dims(current_patch, axis=-1))

        # 0 belső maszk 1 külső maszk 2 differencia

        prediction = predicted_patch[which_prediction]

        prediction = np.reshape(prediction, [24, 32, 32])

        prediction_array[temporal_boundaries[0]:temporal_boundaries[1],
        16:48,
        16:48] += prediction

        # print(prediction_array[32, 32, 32])

        coverage_array[temporal_boundaries[0]:temporal_boundaries[1],
        :,
        :] += 1

    coverage_array = np.maximum(coverage_array, np.ones(coverage_array.shape))
    # Taking the average prediction value for the pixels
    prediction_array = np.divide(prediction_array, coverage_array)
    # print(prediction_array[32,32,32])
    # Removing the prediction values outside of the CT  
    prediction_array = prediction_array[0:size_Z, 0:size_Y, 0:size_X]

    # The average prediction value is continuous between 0 and 1,   
    # so for the segmentation we have to threshold it   
    prediction_array = (prediction_array > 1 / 2) * 1

    return prediction_array

In [13]:
def mask_from_dicom(contour_file_name):
    ds = pydicom.dcmread(contour_file_name)
    pixels = np.array(ds.pixel_array)
    pixels = np.where(pixels > 0, 1, 0)
    return pixels

In [14]:
def one_vessel(image_path):
    '''
    retrun DL segmentation and original segmentation of one plaque.
    '''
    image_dicom=pydicom.dcmread(image_path)
    image = image_dicom.pixel_array

    dif_prediction = full_cta_model_evaluation(image = image, model = model, z_stride = 12, which_prediction=2) 
    #contour of plaque area, segmented by DL
    mask_1_prediction= full_cta_model_evaluation(image = image, model = model, z_stride = 12, which_prediction=0)
    #contour of inner vessel wall segmented by DL
    mask_2_prediction= full_cta_model_evaluation(image = image, model = model, z_stride = 12, which_prediction=1)  
    #contour of outer vessel wall segmented by DL

    mask_1_path = os.path.join(basic_path, image_name[:-13]+"Contour1.dcm") 
    #if the naming process of the mevislab code is not ovewritten, this loads the relevant files
    mask_2_path = os.path.join(basic_path, image_name[:-13]+"Contour2.dcm")
    mask_1 = mask_from_dicom(mask_1_path) #contour of inner vessel wall of the original segmentation
    mask_2 = mask_from_dicom(mask_2_path) #contour of outer vessel wall of the original segmentation
    dif_test=mask_2-mask_1 #contour of the plaque area of the original segmentation
        
    return image, dif_test, dif_prediction, mask_1_prediction, mask_2_prediction, mask_1, mask_2

In [ ]:
image, dif_test, dif_prediction, mask_1_prediction, mask_2_prediction, mask_1, mask_2 = one_vessel(image_path)